## Clase 4, 2020-07-04

### 1. Ejercicio SQL

In [2]:
import sqlite3
import pandas as pd

In [9]:
dbc = sqlite3.connect('./chinook.db')

In [17]:
dbcur = dbc.cursor()

In [24]:
dbcur.execute("SELECT name FROM sqlite_master WHERE type='table';")
dbcur.fetchall()

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

In [40]:
pd.read_sql_query('select * from tracks;', dbc).columns

Index(['TrackId', 'Name', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer',
       'Milliseconds', 'Bytes', 'UnitPrice'],
      dtype='object')

In [39]:
pd.read_sql_query('select * from artists;', dbc).columns

Index(['ArtistId', 'Name'], dtype='object')

In [38]:
pd.read_sql_query('select * from albums;', dbc).columns

Index(['AlbumId', 'Title', 'ArtistId'], dtype='object')

### Canciones de Iron Maiden

In [70]:
pd.read_sql_query("""
                  SELECT * FROM tracks t, albums a, artists ar 
                  WHERE t.AlbumId = a.AlbumId 
                  AND a.ArtistId = ar.ArtistId 
                  AND ar.Name = 'Iron Maiden';""", dbc)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,AlbumId,Title,ArtistId,ArtistId,Name
0,1201,Different World,94,2,1,None,258692,4383764,0.99,94,A Matter of Life and Death,90,90,Iron Maiden
1,1202,These Colours Don't Run,94,2,1,None,412152,6883500,0.99,94,A Matter of Life and Death,90,90,Iron Maiden
2,1203,Brighter Than a Thousand Suns,94,2,1,None,526255,8721490,0.99,94,A Matter of Life and Death,90,90,Iron Maiden
3,1204,The Pilgrim,94,2,1,None,307593,5172144,0.99,94,A Matter of Life and Death,90,90,Iron Maiden
4,1205,The Longest Day,94,2,1,None,467810,7785748,0.99,94,A Matter of Life and Death,90,90,Iron Maiden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,1409,The Clansman,114,1,1,Steve Harris,539689,21592327,0.99,114,Virtual XI,90,90,Iron Maiden
209,1410,When Two Worlds Collide,114,1,1,Blaze Bayley/David Murray/Steve Harris,377312,15093888,0.99,114,Virtual XI,90,90,Iron Maiden
210,1411,The Educated Fool,114,1,1,Steve Harris,404767,16191616,0.99,114,Virtual XI,90,90,Iron Maiden
211,1412,Don't Look To The Eyes Of A Stranger,114,1,1,Steve Harris,483657,19347584,0.99,114,Virtual XI,90,90,Iron Maiden


In [68]:
pd.read_sql_query("""\
                  SELECT t.name, ar.name, count(pt.playlistid) as playlist_count
                  FROM tracks t, albums a, artists ar, playlist_track pt
                  WHERE t.AlbumId = a.AlbumId
                  AND ar.artistid = a.artistid
                  AND pt.trackid = t.trackid
                  group by pt.trackid
                  order by playlist_count desc
                  limit 5;""", dbc)

,Name,Name,playlist_count
0,Intoitus: Adorate Deum,Alberto Turco & Nova Schola Gregoriana,5
1,"Miserere mei, Deus","Richard Marlow & The Choir of Trinity College,...",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",Wilhelm Kempff,5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",Yo-Yo Ma,5
4,"The Messiah: Behold, I Tell You a Mystery... T...",Scholars Baroque Ensemble,5


### 2. Ejercicio FEED PARSER RSS

In [45]:
import feedparser as fp

In [66]:
parsed = fp.parse('https://www.lavoz.com.ar/rss/tecnologia.xml')
posts = parsed.entries
print(len(posts))
[post.title for post in posts]

30


['Ubisoft lanza el juego marcado para ser un nuevo "battle royal", sólo que urbano y futurista',
 'El primer smartphone enrollable ya se está fabricando: LG se anota primero',
 'Este guante con sensores traduce la lengua de signos a voz en tiempo real',
 'Microsoft muestra el nuevo diseño del menú de Windows: fondo unificado y disponible en temas de color claro y oscuro',
 "Una campaña de 'malware' envía SMS falsos a usuarios de Android para robar datos",
 'Lo que se viene en WhatsApp: stickers animados, códigos QR y más',
 'Huawei y Samsung acaparan el mercado mundial de smartphones',
 "Duró poco: Facebook cierra Hobbi, su 'app' experimental similar a Pinterest",
 'Los altavoces portátiles Xboom Gode LG son resistentes al agua y traen una batería de hasta 24 horas',
 'Google Fotos desactiva la copia de seguridad y la sincronización con las carpetas de apps de mensajería como WhatsApp',
 'Facebook lanza una campaña de alfabetización mediática para ayudar a detectar noticias falsas',
 '